In [1]:
import os
import sys
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.insert(0, parent_directory)
import time
import torch
import esm
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from model import MMCLKins
from torch_geometric.data import Data
from process_3dkdavis import generate_inner_coor, generate_protein_sequence, generate_graph_feature, gen_seq_list, gene_smi_fes
from collates import collate
%matplotlib inline

/home/tianyn/.conda/envs/mmclkin/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['N', 'C', 'N', 'C', 'O', 'C', 'C', 'O', 'C', 'O', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S']
['N', 'C', 'N', 'C', 'O', 'C', 'C', 'O', 'C', 'O', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S']
两者相等
['N', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'N', 'N', 'C', 'C', 'C', 'C', 'S', 'O', 'O', 'N', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'S']
['N', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'N', 'N', 'C', 'C', 'C', 'C', 'S', 'O', 'O', 'N', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'S']
两者相等
['N', 'C', 'C', 'C', 'N', 'C', 'C', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'N', 'C', 'N']
['N', 'C', 'C', 'C', 'N', 'C', 'C', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'N', 'C', 'N']
两者相等
['N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C'

#### Downloading the datasets to main directory
##### https://fca_icdb.mpu.edu.mo/DataResource/database/virtual_screening/lrrk2.tar.gz

Running the following script under main directory:
> tar zxvf lrrk2.tar.gz -C ./cases

#### Downloading the pkl file to main directory
##### https://fca_icdb.mpu.edu.mo/DataResource/database/pkls/virtual_screening.tar.gz

Running the following script under main directory:
> tar zxvf virtual_screening.tar.gz -C ./pkls

#### Uploading the datasets

In [2]:
ligand_path = '../cases/lrrk2/sdfs'
kinase_path = '../cases/lrrk2/8fo7_kinase.pdb'
pocket_path = '../cases/lrrk2/8fo7_pocket_12.pdb'

#### Exacting the features

In [3]:
def generate_kin_fes(ki, batch_converter, model1):
    error_pdbs = []
    pdbs_1022 = []
    error_num = []
    kinase_feas = {}
    kina = ki.split('.p')[0]
    pdb_path = ki
    pro_index = gen_seq_list(pdb_path)
    protein, protein_graph, coords = generate_graph_feature(pdb_path)
    protein_sequence, chain_num = generate_protein_sequence(pdb_path)
    
    pr_dist, pr_theta, pr_phi, pr_tau = generate_inner_coor(protein.x, protein.node_s, protein.edge_index)
    prot = [(0, protein_sequence[:1022])]
    batch_labels, batch_strs, batch_tokens = batch_converter(prot)  #0（蛋白序号）；Fasta序列，蛋白表征
    with torch.no_grad():
        results = model1(batch_tokens, repr_layers=[33], return_contacts=True)
    pro_token_repre = results["representations"][33]
    n = 0
    if pro_token_repre.shape[1] != protein.node_s.shape[0] + 2 :
        n = n + 1
        pdbs_1022.append(ki)
        error_num.append(pro_token_repre.shape[1] - protein.node_s.shape[0] - 2)
        if pro_token_repre.shape[1] > 1022:
            print(f'{ki} pdb exceeds 1022!!!')
        else:
            error_pdbs.append(ki)
            print(f'{ki} pdb exceeds 1022!!!')
        
    kinase_fea = Data(pro_graphs=protein_graph,
                    pro_index=pro_index,
                    pro_atoms_feats_s=protein.node_s,
                    pro_atoms_feats_v=protein.node_v,
                    pro_coords_feats=protein.x,
                    pro_edges_feats_s=protein.edge_s,
                    pro_edges_feats_v=protein.edge_v,
                    pro_edge_index=protein.edge_index,
                    pro_token_repre=pro_token_repre,
                    pro_fp=protein_sequence,
                    pr_dist=pr_dist,
                    pr_theta=pr_theta,
                    pr_phi=pr_phi,
                    pr_tau=pr_tau)
    kinase_feas[kina] = kinase_fea
            
    return kinase_feas, pdbs_1022, error_pdbs

In [4]:
model1, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()
kinase_fes, kina_1022, error_kina = generate_kin_fes(kinase_path, batch_converter, model1)
pocket_fes, pock_1022, error_pock = generate_kin_fes(pocket_path, batch_converter, model1)

model_name = "DeepChem/ChemBERTa-10M-MLM"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
lig_fes, error_mol = gene_smi_fes(ligand_path, model, tokenizer)

#### Uploading the pkl file of MMCLKin

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
path = '../pkls/virtual_screening/MMCLKin_DTI_pearson_best.pkl'

lstm_dropout = 0.2
alpha = 0.2
num_heads = 2
hidden_dim = 256
dropout_rate = 0.3
model = MMCLKins(lstm_dropout, alpha, num_heads, hidden_dim, dropout_rate, n_head=8, smile_vocab=63, local_rank=device) 
model.load_state_dict(torch.load(path)['model'], strict=True)
model.to(device)

MMCLKins(
  (dropout): Dropout(p=0.3, inplace=False)
  (leakyrelu): LeakyReLU(negative_slope=0.2)
  (relu): ReLU()
  (prelu): PReLU(num_parameters=1)
  (elu): ELU(alpha=1.0)
  (conv): GCNConv(259, 256)
  (conv3): GCNConv(256, 256)
  (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (bilstm): LSTM(256, 256, dropout=0.2, bidirectional=True)
  (self_attention): A_MultiHeadAttention(
    (qkv_proj): Linear(in_features=512, out_features=1536, bias=True)
    (o_proj): Linear(in_features=512, out_features=512, bias=True)
  )
  (out_fc): Linear(in_features=512, out_features=256, bias=True)
  (out_m_fc): Linear(in_features=256, out_features=128, bias=True)
  (output_fc): Linear(in_features=128, out_features=1, bias=True)
  (smi_egnn): EGNNConv(
    (edge_mlp): Sequential(
      (0): Linear(in_features=57, out_features=256, bias=True)
      (1): SiLU()
      (2): Linear(in_features=256, out_features=256, b

#### Predicting the binding affinity between kinase and inhibitors

In [6]:
model.eval()
pred_results = []
lig_keys = lig_fes.keys()
with torch.no_grad():
    c = 0
    for i, na in enumerate(lig_keys):
        lig_feas = lig_fes[na]
        kin_fes = kinase_fes['../cases/lrrk2/8fo7_kinase']
        poc_fes = pocket_fes['../cases/lrrk2/8fo7_pocket_12']
        x_feats = collate(lig_feas, kin_fes, poc_fes)
        x_feats = x_feats.to(device)
        y_pred, g_spo_att, g_spr_att, s_spo_att, s_spr_att, pg_spo_att, pg_spr_att, ps_spo_att, ps_spr_att, atominmol_indexes, subwinsmi_indexes, pocinpro_indexes = model(x_feats)
        re = [na, y_pred.item(), g_spo_att.tolist(), g_spr_att.tolist(), s_spo_att.tolist(), s_spr_att.tolist(), atominmol_indexes[0].tolist(), 
            pg_spo_att.tolist(), pg_spr_att.tolist(), subwinsmi_indexes[0].tolist(), ps_spo_att.tolist(), ps_spr_att.tolist(), pocinpro_indexes[0].tolist()]
        pred_results.append(re)


name = ['compound_id', 'pred_results', 'graph=smi+pocket_attention', 'graph=smi+pocket_in_protein_attention',      
            'sequence=smi+pocket_attention', 'sequence=smi+pocket_in_protein_attention','graph=smi_atominmol_indexes',
            'graph=atom_in_mol+pocket_attention(atominmol_indexes)', 'graph=atom_in_mol+protein_attention(atominmol_indexes)',
            'sequence=smi_subwinsmi_indexes', 'sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes)',
            'sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes)','pocinpro_indexes']
da = pd.DataFrame(columns=name, data=pred_results)
da

,compound_id,pred_results,graph=smi+pocket_attention,graph=smi+pocket_in_protein_attention,sequence=smi+pocket_attention,sequence=smi+pocket_in_protein_attention,graph=smi_atominmol_indexes,graph=atom_in_mol+pocket_attention(atominmol_indexes),graph=atom_in_mol+protein_attention(atominmol_indexes),sequence=smi_subwinsmi_indexes,sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes),sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes),pocinpro_indexes
0,C58208539_P87855178,5.225255,"[[1.3588517904281616, 0.8791529536247253, -1.5...","[[1.3147011995315552, 0.7562714219093323, -1.6...","[[-0.05761390179395676, 0.580176591873169, -0....","[[0.021261079236865044, 0.5373062491416931, 0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.3588517904281616, 0.8791529536247253, -1.5...","[[1.6802301406860352, 0.9360038638114929, -2.3...","[0, 2, 4, 5, 6, 7, 10, 12, 13, 14, 15, 17, 18,...","[[0.6349223852157593, 0.18837738037109375, -0....","[[0.7628128528594971, 0.22691908478736877, -0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,C97360207_P154193005,5.332994,"[[1.024256944656372, -1.374565601348877, 0.763...","[[0.9837762713432312, -1.425899863243103, 0.78...","[[0.8025575280189514, 1.0512511730194092, 1.05...","[[0.7197107672691345, 0.9269863367080688, 0.90...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.024256944656372, -1.374565601348877, 0.763...","[[1.2325160503387451, -1.977505087852478, 0.96...","[0, 2, 5, 8, 9, 10, 12, 13, 14, 15, 16, 19, 21...","[[1.1896008253097534, 1.3516734838485718, 1.26...","[[1.3834487199783325, 1.4825299978256226, 1.44...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,C72392855_P112585735_2,5.031320,"[[1.1408425569534302, 0.7290148735046387, -1.8...","[[1.0947016477584839, 0.5828035473823547, -1.8...","[[0.5339165329933167, 0.9272330403327942, -0.6...","[[0.5321312546730042, 0.8833274841308594, -0.4...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.1408425569534302, 0.7290148735046387, -1.8...","[[1.440962553024292, 0.7537854313850403, -2.49...","[0, 2, 4, 5, 7, 9, 10, 11, 13, 14, 16, 17, 18,...","[[1.0450314283370972, 1.8988921642303467, -1.6...","[[1.3537360429763794, 2.123884916305542, -1.70...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,C63229077_P97107960,6.966242,"[[0.6698253750801086, -1.5546308755874634, 0.3...","[[0.6121381521224976, -1.5562833547592163, 0.3...","[[-0.6350576281547546, 0.005704487208276987, -...","[[-0.48920148611068726, 0.022338245064020157, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.6698253750801086, -1.5546308755874634, 0.3...","[[0.6900210380554199, -2.1524860858917236, 0.3...","[0, 1, 4, 6, 7, 9, 10, 13, 15, 16, 17, 18, 19,...","[[0.00043663126416504383, -2.2897891998291016,...","[[0.044278450310230255, -2.3186357021331787, 0...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,C08587290_P56887100,5.766104,"[[1.0733104944229126, -0.9794756174087524, -1....","[[1.0099905729293823, -1.1474981307983398, -1....","[[-0.4925158619880676, 1.199284315109253, 0.20...","[[-0.3705075681209564, 1.0187633037567139, 0.2...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.0733104944229126, -0.9794756174087524, -1....","[[1.3708324432373047, -1.5789047479629517, -1....","[0, 2, 4, 5, 8, 10, 12, 14, 15, 16, 17, 18, 21...","[[1.2518633604049683, -0.9771723747253418, 1.0...","[[1.417743444442749, -0.944609522819519, 1.111...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,C01349876_P79935932,5.009304,"[[1.2874572277069092, 0.8629838824272156, -2.9...","[[1.2182140350341797, 0.7210394144058228, -2.8...","[[-0.2330840528011322, 0.924898624420166, 0.70...","[[-0.1541353464126587, 0.8708891272544861, 0.6...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.2874572277069092, 0.8629838824272156, -2.9...","[[1.6063071489334106, 0.9394823312759399, -3.9...","[0, 1, 3, 4, 5, 6, 7, 9, 11, 13, 14,

#### Processing the results

In [7]:
import operator
da['compound_id'] = da['compound_id'].str.replace('_P', '', regex=False)
df = da.sort_values(by='pred_results', ascending=False)
name = list(df['compound_id'])
label = 'True'
all_le = 11
na = []
nas = []
nass = []
for i, ns in enumerate(name):
    if '_' in ns:
        if ns.split('_')[0] not in nas:
            nas.append(ns.split('_')[0])
            nass.append(df.loc[i].tolist())
    else:
        if ns not in nas:
            nas.append(ns)
            nass.append(df.loc[i].tolist())

all_length = len(nas)
m1 = m2 = m3 = m4 = m5 = m6 = m7 = m8 = m9= m10 = 0
for i, com in enumerate(nas):
    if i < all_length*0.01:
        if str(com[:4]) == label:
            m1 = m1 + 1
    if i < all_length*0.02:
        if str(com[:4]) == label:
            m2 = m2 + 1
    if i < all_length*0.03:
        if str(com[:4]) == label:
            m3 = m3 + 1
    if i < all_length*0.04:
        if str(com[:4]) == label:
            m4 = m4 + 1
    if i < all_length*0.05:
        if str(com[:4]) == label:
            m5 = m5 + 1
    if i < all_length*0.06:
        if str(com[:4]) == label:
            m6 = m6 + 1
    if i < all_length*0.07:
        if str(com[:4]) == label:
            m7 = m7 + 1
    if i < all_length*0.08:
        if str(com[:4]) == label:
            m8 = m8 + 1
    if i < all_length*0.09:
        if str(com[:4]) == label:
            m9 = m9 + 1
    if i < all_length*0.1:
        if str(com[:4]) == label:
            m10 = m10 + 1        

m1r = m1/(all_le)
m2r = m2/(all_le)
m3r = m3/(all_le)
m4r = m4/(all_le)
m5r = m5/(all_le)
m6r = m6/(all_le)
m7r = m7/(all_le)
m8r = m8/(all_le)
m9r = m9/(all_le)
m10r = m10/(all_le)

name = [ 'compound_id', 'pred_results', 'graph=smi+pocket_attention',
        'graph=smi+pocket_in_protein_attention','sequence=smi+pocket_attention',
        'sequence=smi+pocket_in_protein_attention',	'graph=smi_atominmol_indexes',
        'graph=atom_in_mol+pocket_attention(atominmol_indexes)','graph=atom_in_mol+protein_attention(atominmol_indexes)',
        'sequence=smi_subwinsmi_indexes', 'sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes)',	
        'sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes)', 'pocinpro_indexes']

new_csv = pd.DataFrame(columns=name, data=nass)
new_csv = pd.concat([new_csv, pd.Series(f'the number of the active drugs in top 1% ::: {m1} / {all_le} == {m1r}'), 
                    pd.Series(f'the number of the active drugs in top 2% ::: {m2} / {all_le} == {m2r}'), 
                    pd.Series(f'the number of the active drugs in top 3% ::: {m3} / {all_le} == {m3r}'), 
                    pd.Series(f'the number of the active drugs in top 4% ::: {m4} / {all_le} == {m4r}'),
                    pd.Series(f'the number of the active drugs in top 5% ::: {m5} / {all_le} == {m5r}'), 
                    pd.Series(f'the number of the active drugs in top 6% ::: {m6} / {all_le} == {m6r}'), 
                    pd.Series(f'the number of the active drugs in top 7% ::: {m7} / {all_le} == {m7r}'), 
                    pd.Series(f'the number of the active drugs in top 8% ::: {m8} / {all_le} == {m8r}'),
                    pd.Series(f'the number of the active drugs in top 9% ::: {m9} / {all_le} == {m9r}'),
                    pd.Series(f'the number of the active drugs in top 10% ::: {m10} / {all_le} == {m10r}')], ignore_index=True)
save_path = '../output/vs_lrrk2/'
os.makedirs(save_path, exist_ok=True)
new_csv.to_csv(f'{save_path}/vs_lrrk2.csv', encoding='gbk', index=False)
    
new_csv


,compound_id,pred_results,graph=smi+pocket_attention,graph=smi+pocket_in_protein_attention,sequence=smi+pocket_attention,sequence=smi+pocket_in_protein_attention,graph=smi_atominmol_indexes,graph=atom_in_mol+pocket_attention(atominmol_indexes),graph=atom_in_mol+protein_attention(atominmol_indexes),sequence=smi_subwinsmi_indexes,sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes),sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes),pocinpro_indexes,0
0,C5820853987855178,5.225255,"[[1.3588517904281616, 0.8791529536247253, -1.5...","[[1.3147011995315552, 0.7562714219093323, -1.6...","[[-0.05761390179395676, 0.580176591873169, -0....","[[0.021261079236865044, 0.5373062491416931, 0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.3588517904281616, 0.8791529536247253, -1.5...","[[1.6802301406860352, 0.9360038638114929, -2.3...","[0, 2, 4, 5, 6, 7, 10, 12, 13, 14, 15, 17, 18,...","[[0.6349223852157593, 0.18837738037109375, -0....","[[0.7628128528594971, 0.22691908478736877, -0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
1,C97360207154193005,5.332994,"[[1.024256944656372, -1.374565601348877, 0.763...","[[0.9837762713432312, -1.425899863243103, 0.78...","[[0.8025575280189514, 1.0512511730194092, 1.05...","[[0.7197107672691345, 0.9269863367080688, 0.90...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.024256944656372, -1.374565601348877, 0.763...","[[1.2325160503387451, -1.977505087852478, 0.96...","[0, 2, 5, 8, 9, 10, 12, 13, 14, 15, 16, 19, 21...","[[1.1896008253097534, 1.3516734838485718, 1.26...","[[1.3834487199783325, 1.4825299978256226, 1.44...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
2,C72392855112585735_2,5.031320,"[[1.1408425569534302, 0.7290148735046387, -1.8...","[[1.0947016477584839, 0.5828035473823547, -1.8...","[[0.5339165329933167, 0.9272330403327942, -0.6...","[[0.5321312546730042, 0.8833274841308594, -0.4...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.1408425569534302, 0.7290148735046387, -1.8...","[[1.440962553024292, 0.7537854313850403, -2.49...","[0, 2, 4, 5, 7, 9, 10, 11, 13, 14, 16, 17, 18,...","[[1.0450314283370972, 1.8988921642303467, -1.6...","[[1.3537360429763794, 2.123884916305542, -1.70...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
3,C0858729056887100,5.766104,"[[1.0733104944229126, -0.9794756174087524, -1....","[[1.0099905729293823, -1.1474981307983398, -1....","[[-0.4925158619880676, 1.199284315109253, 0.20...","[[-0.3705075681209564, 1.0187633037567139, 0.2...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.0733104944229126, -0.9794756174087524, -1....","[[1.3708324432373047, -1.5789047479629517, -1....","[0, 2, 4, 5, 8, 10, 12, 14, 15, 16, 17, 18, 21...","[[1.2518633604049683, -0.9771723747253418, 1.0...","[[1.417743444442749, -0.944609522819519, 1.111...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
4,C2714203940001166_2,4.985216,"[[0.8558474779129028, -0.7431292533874512, -0....","[[0.8581149578094482, -0.8063104152679443, -0....","[[0.283502459526062, 1.1952871084213257, -0.25...","[[0.2762032151222229, 1.0496724843978882, -0.1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.8558474779129028, -0.7431292533874512, -0....","[[1.0474354028701782, -1.192429542541504, -1.0...","[0, 2, 4, 5, 7, 8, 9, 10, 12, 15, 17, 18, 19, ...","[[1.3542075157165527, 0.4794228672981262, -0.6...","[[1.5382024049758911, 0.578571081161499, -0.56...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 6% ::: 8...
717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 7% ::: 8...
718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 8% ::: 9...
719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 9% ::

In [8]:
f'the number of the active drugs in top 1% ::: {m1} / {all_le} == {m1r}\
the number of the active drugs in top 2% ::: {m2} / {all_le} == {m2r}\
the number of the active drugs in top 3% ::: {m3} / {all_le} == {m3r}\
the number of the active drugs in top 4% ::: {m4} / {all_le} == {m4r}\
the number of the active drugs in top 5% ::: {m5} / {all_le} == {m5r}\
the number of the active drugs in top 6% ::: {m6} / {all_le} == {m6r}\
the number of the active drugs in top 7% ::: {m7} / {all_le} == {m7r}\
the number of the active drugs in top 8% ::: {m8} / {all_le} == {m8r}\
the number of the active drugs in top 9% ::: {m9} / {all_le} == {m9r}\
the number of the active drugs in top 10% ::: {m10} / {all_le} == {m10r}'

'the number of the active drugs in top 1% ::: 5 / 11 == 0.45454545454545453the number of the active drugs in top 2% ::: 6 / 11 == 0.5454545454545454the number of the active drugs in top 3% ::: 7 / 11 == 0.6363636363636364the number of the active drugs in top 4% ::: 8 / 11 == 0.7272727272727273the number of the active drugs in top 5% ::: 8 / 11 == 0.7272727272727273the number of the active drugs in top 6% ::: 8 / 11 == 0.7272727272727273the number of the active drugs in top 7% ::: 8 / 11 == 0.7272727272727273the number of the active drugs in top 8% ::: 9 / 11 == 0.8181818181818182the number of the active drugs in top 9% ::: 9 / 11 == 0.8181818181818182the number of the active drugs in top 10% ::: 9 / 11 == 0.8181818181818182'